

# Project: Comparison of Existing Algorithms applied on Flappy Bird

<a href="https://github.com/lplacidet/evolution_project">https://github.com/lplacidet/evolution_project</a>

Importing Libraries

In [1]:
using Random
using LinearAlgebra
using Statistics
using Plots

using Cxx, Libdl

gr(reuse=true)

Plots.GRBackend()

In [2]:
const path_to_lib = pwd()
addHeaderDir(path_to_lib*"/world/", kind=C_System)
cxxinclude("runWorld.h")
print("gg")
#objective = @cxx FlappyXP->evaluate
print("re-gg")

ggre-gg

In [3]:
N_genes = 5

mutable struct Individu
    genes::Array{Float64}
    fitness::Float64
end

function Individu(n::Int)
    newgenes = zeros(n)
    for i in 1:n
        newgenes[i] = rand()
    end
    Individu(newgenes, -Inf)
end 

Individu(N_genes)

Individu([0.33212316818130105, 0.6011006530874785, 0.14648335448208893, 0.05077129244440437, 0.8976263109017719], -Inf)

In [4]:
cxx"""

static double evaluate_bird(float g1, float g2, float g3, float g4, float g5) {
float ind[5] = {g1,g2,g3,g4,g5};
bool dbg = false;
#ifdef DISPLAY
	initscr();
	timeout(0);
	noecho();
	curs_set(FALSE);
#endif
	const int NRUN = 3;
	auto &g = ind;
	double d = 0;
	for (int r = 0; r < NRUN; ++r) {
		World world;
		world.gen.seed(r * 100);
		while (world.bestiau.vivant) {
#ifdef DISPLAY
            std::this_thread::sleep_for(std::chrono::milliseconds(25));
            display(world, world.dist == 0);
#endif
            world.update();
            // update inputs
            float birdY = world.bestiau.y;
            float birdVY = world.bestiau.vit;
            float obsX;
            float obsY;
            if (world.obstacles.size() > 0) {
                const auto &o = world.obstacles.front();
                obsY = o.y + world.hauteurPassage * 0.5;
                obsX = o.x / world.W;
            } else {
                obsY = world.H * 0.5;
                obsX = 1000.0;
            }
            if ((g[0]*birdY+g[1]*birdVY+g[2]*obsX+g[3]*obsY+g[4]*world.dist)>1.0) world.bestiauUp();
        }
        d += world.dist;
        float distToTunnel = world.bestiau.y-(world.obstacles.front().y+world.hauteurPassage*0.5);
        d += 0.3*(1-((distToTunnel*distToTunnel)/(world.H*world.H)));
    }
#ifdef DISPLAY
    endwin();
#endif
    return d;
}
"""

true

In [5]:
cxx"""
#define DISPLAY
#include <chrono>
#include "coverage.hpp"
#include "include/gaga/gaga.hpp"
#include "include/grgen/multiint.hpp"
#include "include/grgen/real.hpp"
#include "include/grgen/grn.hpp"
#include "signalxp.hpp"
#include "bestiau.hpp"
#include <curses.h>

template <typename W> void display(const W &w, bool needClear = false) {
	start_color();
	init_color(COLOR_GREEN, 200, 800, 0);
	init_color(COLOR_RED, 900, 200, 0);
	init_color(COLOR_BLUE, 0, 450, 800);
	double squareV = pow(1.0 + 10.0 * w.bestiau.vit, 2);
	init_color(COLOR_YELLOW, 600 + 100 * squareV, 600 - 100 * squareV,
	           100 + (squareV * 0.2));
	init_pair(1, COLOR_WHITE, COLOR_BLUE);
	init_pair(2, COLOR_GREEN, COLOR_GREEN);
	init_pair(3, COLOR_RED, COLOR_RED);
	init_pair(4, COLOR_YELLOW, COLOR_YELLOW);
	bkgd(COLOR_PAIR(1));

	int row;
	int col;
	getmaxyx(stdscr, row, col);
	double whRatio = w.W / w.H;
	double crRatio = (double)col / (double)row;
	double coef = crRatio < whRatio ? (double)col / w.W : (double)row / w.H;

	if (needClear) {
		clear();
	}

	// bordures
	std::string bordures(w.W * coef, ' ');
	attron(COLOR_PAIR(2));
	for (size_t i = 0; i < w.epaisseurBordures * coef - 1; ++i) {
		mvprintw(i, 0, bordures.c_str());
	}
	for (size_t i = 0; i < w.epaisseurBordures * coef; ++i) {
		mvprintw(w.H * coef - w.epaisseurBordures * coef + i, 0, bordures.c_str());
	}
	attroff(COLOR_PAIR(2));
	std::string clearObstacles(w.largeurObstacles * coef + 10, ' ');
	attron(COLOR_PAIR(1));
	for (size_t r = w.epaisseurBordures * coef; r < (w.H - w.epaisseurBordures) * coef - 2;
	     ++r) {
		mvprintw(r, 0, clearObstacles.c_str());
	}
	std::string strObstacles(w.largeurObstacles * coef, ' ');
	for (auto &o : w.obstacles) {
		if (o.x < w.W - w.largeurObstacles) {
			for (size_t r = w.epaisseurBordures * coef;
			     r < (w.H - w.epaisseurBordures) * coef - 2; ++r) {
				attron(COLOR_PAIR(1));
				mvprintw(r, o.x * coef, clearObstacles.c_str());
				attron(COLOR_PAIR(3));
				if (r < coef * (w.H - (o.y + w.hauteurPassage)) || r > coef * (w.H - o.y))
					mvprintw(r, o.x * coef, strObstacles.c_str());
			}
		}
	}
	attroff(COLOR_PAIR(3));
	// bestiau
	double deltaVit = 0.05;
	attron(COLOR_PAIR(4));
	mvprintw(coef * (w.H - w.bestiau.y), coef * w.bestiau.x, "  ");
	attron(COLOR_PAIR(1));
	mvprintw(0.1 * coef, 2.6 * coef, "%f", w.dist);
	attroff(COLOR_PAIR(1));
	refresh();
}

static double evaluate_bird_display(float g1, float g2, float g3, float g4, float g5) {
float ind[5] = {g1,g2,g3,g4,g5};
bool dbg = false;
#ifdef DISPLAY
	initscr();
	timeout(0);
	noecho();
	curs_set(FALSE);
#endif
	const int NRUN = 3;
	auto &g = ind;
	double d = 0;
	for (int r = 0; r < NRUN; ++r) {
		World world;
		world.gen.seed(r * 100);
		while (world.bestiau.vivant) {
#ifdef DISPLAY
            std::this_thread::sleep_for(std::chrono::milliseconds(25));
            display(world, world.dist == 0);
#endif
            world.update();
            // update inputs
            float birdY = world.bestiau.y;
            float birdVY = world.bestiau.vit;
            float obsX;
            float obsY;
            if (world.obstacles.size() > 0) {
                const auto &o = world.obstacles.front();
                obsY = o.y + world.hauteurPassage * 0.5;
                obsX = o.x / world.W;
            } else {
                obsY = world.H * 0.5;
                obsX = 1000.0;
            }
            if ((g[0]*birdY+g[1]*birdVY+g[2]*obsX+g[3]*obsY+g[4]*world.dist)>1.0) world.bestiauUp();
        }
        d += world.dist;
        float distToTunnel = world.bestiau.y-(world.obstacles.front().y+world.hauteurPassage*0.5);
        d += 0.3*(1-((distToTunnel*distToTunnel)/(world.H*world.H)));
    }
#ifdef DISPLAY
    endwin();
#endif
    return d;
}
"""

true

In [6]:
display_bird(g1::Float64, g2::Float64, g3::Float64, g4::Float64, g5::Float64) = @cxx evaluate_bird_display(g1,g2,g3,g4,g5)

display_bird (generic function with 1 method)

In [ ]:
display_bird(0.1,0.2,0.3,0.4,0.5)

In [ ]:
cpptojulia(g1::Float64, g2::Float64, g3::Float64, g4::Float64, g5::Float64) = @cxx evaluate_bird(g1,g2,g3,g4,g5)

In [ ]:
function evaluate_bird(g::Array{Float64})
    -1*cpptojulia(g[1],g[2],g[3],g[4],g[5])
end

In [ ]:
test_ind = Individu(N_genes)
println(test_ind)
cpptojulia(test_ind.genes[1], test_ind.genes[2],test_ind.genes[3], test_ind.genes[4], test_ind.genes[5])



## I. CMA-ES Algorithm


In [ ]:
mutable struct CMAES
    N::Int
    μ::Int
    λ::Int
    τ::Float64
    τ_c::Float64
    τ_σ::Float64
    population::Array{Array{Float64}}
    offspring::Array{Array{Float64}}
    F_μ::Array{Float64}
    F_λ::Array{Float64}
    C::Array{Float64}
    s::Array{Float64}
    s_σ::Array{Float64}
    σ::Float64
    E::Array{Float64}
    W::Array{Float64}
    x::Array{Float64}
end

In [ ]:
N_value = N_genes
mu = 5
lambda = 5
objective = evaluate_bird

In [ ]:
function CMAES(;N=N_value, μ=mu, λ=lambda, τ=sqrt(N), τ_c=N^2, τ_σ=sqrt(N))
    x = randn(N)
    population = fill(x, µ)
    offspring = Array{Array{Float64}}(undef, λ)
    F_µ = Inf .* ones(µ)
    F_λ = Inf .* ones(λ)
    C = Array(Diagonal{Float64}(I, N))
    s = zeros(N)
    s_σ = zeros(N)
    σ = 0.1
    E = zeros(N, λ)
    W = zeros(N, λ);
    CMAES(N, μ, λ, τ, τ_c, τ_σ, population, offspring, F_µ, F_λ, C, s, s_σ, σ, E, W, x)
end

In [ ]:
function step_cmaes!(c::CMAES; obj=objective, visualize=false, anim=Nothing)
    # L1
    sqrt_c = cholesky((c.C + c.C') / 2.0).U
    for i in 1:c.λ
        c.E[:,i] = randn(c.N)
        c.W[:,i] = c.σ * (sqrt_c * c.E[:,i])
        c.offspring[i] = c.x + c.W[:,i]
        c.F_λ[i] =  obj(c.offspring[i])
    end    
    # Select new parent population
    idx = sortperm(c.F_λ)[1:c.μ]
    for i in 1:c.μ
        c.population[i] = c.offspring[idx[i]]
        c.F_μ[i] = c.F_λ[idx[i]]
    end    
    # L2
    w = vec(mean(c.W[:,idx], dims=2))
    c.x += w 
    # L3
    c.s = (1.0 - 1.0/c.τ)*c.s + (sqrt(c.μ/c.τ * (2.0 - 1.0/c.τ))/c.σ)*w   
    # L4
    c.C = (1.0 - 1.0/c.τ_c).*c.C + (c.s./c.τ_c)*c.s'    
    # L5
    ɛ = vec(mean(c.E[:,idx], dims=2))
    c.s_σ = (1.0 - 1.0/c.τ_σ)*c.s_σ + sqrt(c.μ/c.τ_σ*(2.0 - 1.0/c.τ_σ))*ɛ    
    # L6
    c.σ = c.σ*exp(((c.s_σ'*c.s_σ)[1] - c.N)/(2*c.N*sqrt(c.N)))
    #println("this is old sigma ---> ",c.σ)
    if c.σ > 1.0
        c.σ = 1.0
        print(" -cs") #stands for "capping sigma"
    end
    #println("this is new sigma ---> ",c.σ)
    if visualize
        plot(xs, ys, fz, st=:contour)
        scatter!([c.offspring[i][1] for i in 1:λ], [c.offspring[i][2] for i in 1:λ], 
            xlims=(-5, 5), ylims=(-5, 5), legend=:none)
        scatter!([c.x[1]], [c.x[2]], color=:black, marker=:rect,
            xlims=(-5, 5), ylims=(-5, 5), legend=:none)
        frame(anim)
    end
    c
end

In [ ]:
function plot_obj_cmaes()
    c = CMAES()
    println("x initial: ", c.x)
    anim = Animation()
    for i in 1:100
        v = mod(i, 1) == 0
        step_cmaes!(c, visualize=v, anim=anim)
    end
    println("x final: ", c.x)
    gif(anim)
end

In [ ]:
function not_plot_obj_cmaes(nbgen::Int)
    freq = 1 #frequence d'echantillonage du graphique
    freq_purcent = 5 #afficher complétion tous les n%
    c = CMAES()
    gen = zeros(Int(nbgen/freq))
    fitness = zeros(Int(nbgen/freq))
    next_purcentage = freq_purcent
    for i in 1:nbgen
        v = mod(i, 1) == 0
        if i%freq==0
            gen[Int(i/freq)] = i
            fitness[Int(i/freq)] = -1*evaluate_bird(c.population[1])
        end
        step_cmaes!(c, visualize=false)
        if i/nbgen >= next_purcentage/100
            println(next_purcentage,"% done ...", c.σ)
            next_purcentage += freq_purcent
        end
    end
    c.population[1], gen, fitness
end

In [ ]:
fz(x, y) = objective([x, y])
my_guess, liste_generations, liste_fitness = not_plot_obj_cmaes(1000)

println(-1*evaluate_bird(my_guess))
println(my_guess)

plot(liste_generations, liste_fitness, title = "Evolution de la fitness pour CMA-ES")
xlabel!("Génération")
ylabel!("Fitness du meilleur individu")



## II. ES (mu/signam, lambda)


In [ ]:
npop = 50     # population size
sigma = 0.1   # noise standard deviation
alpha = 0.001 # step size
x = randn(2)  # initial expert

In [ ]:
xs = -5.0:0.1:5.0
ys = -5.0:0.1:5.0
#fz(x, y) = objective([x, y]);

function step_es(x::Array{Float64}; npop=50, sigma=0.1, alpha=0.01, visualize=false, anim=Nothing)
    N = randn(npop, 2)
    P = repeat(x, 1, npop)' .+ sigma .* N
    R = zeros(npop)
    for i in eachindex(R)
        R[i] = objective(P[i, :]) #evaluation
    end
    A = (R .- mean(R)) ./ std(R) #the selection is done here: we are giving a higher porbability to individuals that did well
    
    if visualize
        plot(xs, ys, fz, st=:contour)
        scatter!(P[:, 1], P[:, 2], xlims=(-5, 5), ylims=(-5, 5), zcolor=R)
        scatter!([x[1]], [x[2]], legend=:none, color=:black, marker=:rect)
        frame(anim)
    end
    
    x .+ alpha/(npop * sigma) .* [dot(N[:, i], A) for i in 1:size(N, 2)] #modification step: changing the center of gravity
end

In [ ]:
function plot_obj_es()
    x = randn(2) #initial population: which is a single point
    println("x initial: ", x)
    anim = Animation()
    for i in 1:500
        v = mod(i, 10) == 0
        x = step_es(x, npop=50, sigma=0.1, alpha=0.001, visualize=v, anim=anim)
    end
    println("x final: ", x)
    gif(anim)
end

In [ ]:
xs = -5.0:0.1:5.0
ys = -5.0:0.1:5.0

#fz(x, y) = objective_es([x, y])
println(solution) # optimal for sphere and rastrigin
#plot_obj_cmaes()